In [29]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#  2021년 지역별 교통사고 데이터 분석 및 시각화
* 어느 지역이 사망교통사고가 많이 발생했었는지  (? 경기도)
* 어느 시간대에 사망교통사고가 많이 발생 했었는지 (? 17시)
* 우리집 주변에 사망 교통사고가 발생 했었는지 화인
* 대전에서는 어느 구에서 가장 사망 교통사고가 발생했었는지 확인 (?유성구?서구?)


## 1.데이터 준비


In [30]:
!pip install koreanize-matplotlib

In [31]:
import matplotlib as plt
import seaborn as sns
import plotly.express as px
import koreanize_matplotlib

#지도 시각화
import folium

#파이썬 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

In [32]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/abc부트캠프/visual/2교통사고분석/도로교통공단_사망 교통사고 정보_20211231.csv',encoding='EUC-KR')
df.head()


,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,...,사고유형,가해자법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X(UTMK),발생위치Y(UTMK),경도,위도
0,2021,2021-01-01 03:00,야,금,1,3,0,3,0,경북,...,추돌,안전운전 의무 불이행,교차로,교차로부근,승용차,승용차,1097010.0,1793385.0,128.578152,36.132653
1,2021,2021-01-01 09:00,주,금,1,0,0,0,0,충남,...,공작물충돌,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,902369.0,1847109.0,126.408201,36.616845
2,2021,2021-01-01 15:00,주,금,1,0,0,0,0,강원,...,측면충돌,안전운전 의무 불이행,교차로,교차로내,원동기장치자전거,승용차,1123975.0,1974509.0,128.907484,37.761842
3,2021,2021-01-01 19:00,야,금,1,0,0,0,0,전남,...,횡단중,안전운전 의무 불이행,단일로,기타단일로,화물차,보행자,886507.0,1613961.0,126.263573,34.513391
4,2021,2021-01-01 21:00,야,금,1,0,0,0,0,경기,...,기타,기타,단일로,기타단일로,승용차,보행자,953522.0,1915403.0,126.976011,37.236327


### 데이터 확인하기

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   발생년          2816 non-null   int64  
 1   발생년월일시       2816 non-null   object 
 2   주야           2816 non-null   object 
 3   요일           2816 non-null   object 
 4   사망자수         2816 non-null   int64  
 5   부상자수         2816 non-null   int64  
 6   중상자수         2816 non-null   int64  
 7   경상자수         2816 non-null   int64  
 8   부상신고자수       2816 non-null   int64  
 9   발생지시도        2816 non-null   object 
 10  발생지시군구       2816 non-null   object 
 11  사고유형_대분류     2816 non-null   object 
 12  사고유형_중분류     2816 non-null   object 
 13  사고유형         2816 non-null   object 
 14  가해자법규위반      2816 non-null   object 
 15  도로형태_대분류     2816 non-null   object 
 16  도로형태         2816 non-null   object 
 17  가해자_당사자종별    2816 non-null   object 
 18  피해자_당사자종별    2816 non-null   object 
 19  발생위치X(

##2.사망교통사고 데이터 전처리

원본데이터 복사하기

In [34]:
accident_df=df.copy()
accident_df.head()

,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,...,사고유형,가해자법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X(UTMK),발생위치Y(UTMK),경도,위도
0,2021,2021-01-01 03:00,야,금,1,3,0,3,0,경북,...,추돌,안전운전 의무 불이행,교차로,교차로부근,승용차,승용차,1097010.0,1793385.0,128.578152,36.132653
1,2021,2021-01-01 09:00,주,금,1,0,0,0,0,충남,...,공작물충돌,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,902369.0,1847109.0,126.408201,36.616845
2,2021,2021-01-01 15:00,주,금,1,0,0,0,0,강원,...,측면충돌,안전운전 의무 불이행,교차로,교차로내,원동기장치자전거,승용차,1123975.0,1974509.0,128.907484,37.761842
3,2021,2021-01-01 19:00,야,금,1,0,0,0,0,전남,...,횡단중,안전운전 의무 불이행,단일로,기타단일로,화물차,보행자,886507.0,1613961.0,126.263573,34.513391
4,2021,2021-01-01 21:00,야,금,1,0,0,0,0,경기,...,기타,기타,단일로,기타단일로,승용차,보행자,953522.0,1915403.0,126.976011,37.236327


### 날짜 컬럼에서 시간 파생 컬럼 만들기

In [35]:
accident_df['발생년월일시'].head()

,발생년월일시
0,2021-01-01 03:00
1,2021-01-01 09:00
2,2021-01-01 15:00
3,2021-01-01 19:00
4,2021-01-01 21:00


In [36]:
#발생년월일시 데이터 타입 변경(문자열 -> 날짜)
#YYYY-MM-DD H:M:S
accident_df['발생년월일시']=pd.to_datetime(accident_df['발생년월일시'],
               format='%Y-%m-%d %H:%M', #날짜 유형
               errors='raise'   #ignore 무시 raise발생시켜라
              )
accident_df['발생년월일시'].head()

,발생년월일시
0,2021-01-01 03:00:00
1,2021-01-01 09:00:00
2,2021-01-01 15:00:00
3,2021-01-01 19:00:00
4,2021-01-01 21:00:00


In [48]:
# 파생변수 → 발생년월일, 발생월, 발생시간
accident_df['발생년월일']=accident_df['발생년월일시'].dt.date
accident_df['발생월']=accident_df['발생년월일시'].dt.month
accident_df['발생시간']=accident_df['발생년월일시'].dt.hour
accident_df.iloc[:5,-3:]

,발생년월일,발생월,발생시간
0,2021-01-01,1,3
1,2021-01-01,1,9
2,2021-01-01,1,15
3,2021-01-01,1,19
4,2021-01-01,1,21


### 전저처리 데이터 저장

In [57]:
accident_df.to_csv('2021년사망교통사고 전처리.csv',encoding='utf-8-sig',index=False)

In [60]:
df1=pd.read_csv('2021년사망교통사고 전처리.csv')
df1.head(2)

,발생년,발생년월일시,주야,요일,사망자수,부상자수,중상자수,경상자수,부상신고자수,발생지시도,...,도로형태,가해자_당사자종별,피해자_당사자종별,발생위치X(UTMK),발생위치Y(UTMK),경도,위도,발생년월일,발생월,발생시간
0,2021,2021-01-01 03:00:00,야,금,1,3,0,3,0,경북,...,교차로부근,승용차,승용차,1097010.0,1793385.0,128.578152,36.132653,2021-01-01,1,3
1,2021,2021-01-01 09:00:00,주,금,1,0,0,0,0,충남,...,기타단일로,승용차,없음,902369.0,1847109.0,126.408201,36.616845,2021-01-01,1,9


## 3.분석 및 시각화

### 날짜 /시간

In [62]:
#x축 날싸, y축 사고
fig=px.scatter(accident_df,
               x='발생년월일',
               y='발생시간',
               size='사망자수',   #점의 크기를 '사망자'로 지정->버블차트
               color='발생지시도'
               )
fig.show()

### 시간대별

In [66]:
fig_time=px.bar(accident_df, x='발생시간',y='사망자수')
fig_time.show()

In [68]:
fig_time2=px.bar(accident_df, y='발생시간',x='사망자수',orientation='h')
fig_time2.show()

### 시도별

In [71]:
accident_df['발생지시도'].unique()

array(['경북', '충남', '강원', '전남', '경기', '광주', '서울', '경남', '제주', '부산', '대전',
       '대구', '인천', '전북', '세종', '충북', '울산'], dtype=object)

In [87]:
area_list=accident_df['발생지시도'].unique().sort()
print(area_list)

None


In [90]:
area_list=accident_df['발생지시도'].unique()
area_list.sort()
area_list

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [91]:
fig_area=px.histogram(accident_df,
                      x='사망자수',
                      y='발생지시도',
                      orientation='h', #가로방향
                      text_auto=True,   #숫자 표시
                      category_orders={'발생지시도':area_list} #정렬
                      )
fig_area.show()

In [95]:
# 사망교통사고 시도별 내림차순
area_desc=accident_df['발생지시도'].value_counts().index
fig_area=px.histogram(accident_df,
                      x='사망자수',
                      y='발생지시도',
                      orientation='h', #가로방향
                      text_auto=True,   #숫자 표시
                      category_orders={'발생지시도':area_desc} #정렬
                      )

fig_area.show()
#충남 경남 전남 전북 정렬이 제대로 이뤄지지 않은 이유

### 요일별

In [97]:
day_list=['월','화','수','목','금','토','일']

In [98]:
fig_area=px.histogram(accident_df,
                      x='사망자수',
                      y='요일',
                      orientation='h', #가로방향
                      text_auto=True,   #숫자 표시
                      category_orders={'요일':day_list}

                      )
fig_area.show()

### 사고유형별

In [104]:

fig_area=px.histogram(accident_df,
                      x='사망자수',
                      y='사고유형',
                      orientation='h', #가로방향
                      text_auto=True,   #숫자 표시
                      category_orders={'사고유형':accident_df['사고유형'].value_counts().index} #정렬
                      )
fig_area.show()

In [119]:
fig_area=px.histogram(accident_df,
                      x='사망자수',
                      y='가해자법규위반',
                      orientation='h', #가로방향
                      text_auto=True,   #숫자 표시
                      category_orders={'가해자법규위반':accident_df['가해자법규위반'].value_counts().index} #정렬
                      )
fig_area.show()

In [111]:
accident_df['가해자법규위반'].value_counts()

,count
가해자법규위반,
안전운전 의무 불이행,1878
기타,339
신호위반,239
중앙선 침범,169
보행자 보호의무 위반,107
안전거리 미확보,45
교차로 통행방법 위반,39


##  4.대전 사망교통사고 분석 및 시각화

In [130]:
dj_df=accident_df.query("발생지시도 == '대전' ")    #큰따옴표안에 작은 따옴표가 들어갈 수 있다
dj_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 44 to 2809
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   발생년          59 non-null     int64         
 1   발생년월일시       59 non-null     datetime64[ns]
 2   주야           59 non-null     object        
 3   요일           59 non-null     object        
 4   사망자수         59 non-null     int64         
 5   부상자수         59 non-null     int64         
 6   중상자수         59 non-null     int64         
 7   경상자수         59 non-null     int64         
 8   부상신고자수       59 non-null     int64         
 9   발생지시도        59 non-null     object        
 10  발생지시군구       59 non-null     object        
 11  사고유형_대분류     59 non-null     object        
 12  사고유형_중분류     59 non-null     object        
 13  사고유형         59 non-null     object        
 14  가해자법규위반      59 non-null     object        
 15  도로형태_대분류     59 non-null     object        
 16  도로형태        

### 발생시간별

In [145]:
#전국 18시,대전 17시
fig_dj=px.histogram(dj_df, x='사망자수', y='발생시간', orientation='h',
                      text_auto=True
                      )
fig_dj.show()

### 대전지역구별

In [143]:

fig_dj=px.histogram(dj_df,
                      x='사망자수',
                      y='발생지시군구',
                      orientation='h', #가로방향
                      text_auto=True
                      )
fig_dj.show()

In [181]:
dj_df=accident_df.query("발생지시도 == '충남' ")
day_list=['월','화','수','목','금','토','일']
fig_dj=px.histogram(dj_df,
                      x='요일',
                      y='사망자수',

                      text_auto=True,   #숫자 표시
                      category_orders={'요일':day_list}
                      )
fig_dj.show()

## 5.지도맵 시가가화

### 기본 지도 준비

In [148]:
map=folium.Map(location=[])  #dn
map

### 대전 사망교통사고 위치 표시

In [150]:
# 사망교통사고 위치에 CircleMarker
#싸클마커 크기= 사망자수+부상자수(중상+경상)
#싸클마커 클릭 시 popup으로 사고 유형 표출

#써클마커 사이즈는 실수형 데이터만 취급(원주율때문이다)
dj_df=dj_df.astype({'사망자수':'float','부상자수':'float'})

In [157]:
#1) 기본 지도 준비
dj_map=folium.Map(location=[ 36.3504119,127.3845475])

# 2)써클마커 : for문으로 dj_df 정보가 있을때까지 마커 add
for i in dj_df.index:
  #마커 사이즈로 사용할 변수cnt
  사이즈=dj_df['사망자수'][i] + dj_df['부상자수'][i]

  #사고 위치
  lat=dj_df['위도'][i]
  lng=dj_df['경도'][i]

  #지도에 추가 표시
  folium.CircleMarker(location=[lat,lng],
                       radius=사이즈*10,
                      popup=dj_df['사고유형'][i],
                      color='blue',
                      fill_color='red').add_to(dj_map)
dj_map

In [170]:
#1) 기본 지도 준비
map=folium.Map(location=[ 36.825687, 127.184699])

# 2)써클마커 : for문으로 dj_df 정보가 있을때까지 마커 add
for i in dj_df.index:
  #마커 사이즈로 사용할 변수cnt
  사이즈=dj_df['사망자수'][i] + dj_df['부상자수'][i]

  #사고 위치
  lat=dj_df['위도'][i]
  lng=dj_df['경도'][i]

  #지도에 추가 표시
  folium.Marker(location=[lat,lng],

                      popup=dj_df['사고유형'][i],

                      icon=folium.Icon(color='blue',icon='fa-car',prefix='fa')
                      ).add_to(map)
map

### 대전 사망교통사고 지도 저장

# 새 섹션

In [168]:
map.save('2021대전 사망교통사고 지도.html')

#기타

In [179]:
주야=px.bar(dj_df,x='사망자수',y='주야', orientation='h',
         category_orders={'주야':dj_df['주야'].value_counts().index})
주야.show()

In [182]:
충남지역=accident_df[accident_df['발생지시도'] == '충남']
시군구=충남지역['발생지시군구'].value_counts().index
충남시별차트=px.histogram(충남지역,
                       x='사망자수',
                       y='발생지시군구',
                       orientation='h', #가로방향
                       text_auto=True,   #숫자 표시
                       category_orders={'발생지시군구':시군구} #정렬
                      )
충남시별차트.update_layout(title='충남시군구')
충남시별차트.show()

도로형태=px.histogram(충남지역,
                  x='사망자수',
                  y='도로형태',
                  orientation='h',
                  text_auto=True,
                  )
도로형태.update_layout(title='21년 충남 도로형태')
도로형태.show()